In [1]:
import numpy as np

In [4]:
import torch

def distance_corr(var_1,var_2,normedweight,power=1):
    """var_1: First variable to decorrelate (eg mass)
    var_2: Second variable to decorrelate (eg classifier output)
    normedweight: Per-example weight. Sum of weights should add up to N (where N is the number of examples)
    power: Exponent used in calculating the distance correlation
    
    va1_1, var_2 and normedweight should all be 1D torch tensors with the same number of entries
    
    Usage: Add to your loss function. total_loss = BCE_loss + lambda * distance_corr
    """
    
    
#     xx = var_1.view(-1, 1).repeat(1, len(var_1)).view(len(var_1),len(var_1))
#     yy = var_1.repeat(len(var_1),1).view(len(var_1),len(var_1))
#     amat = (xx-yy).abs()
#     print(var_1.view(-1, 1))
#     print(amat)
#     print(len(var_1))

    xx = var_1.view(-1, 1)
    #print(xx)
    xx = xx.repeat(1, len(var_1))
    #print(xx)
    xx = xx.view(len(var_1),len(var_1))
    #print(xx)
    yy = var_1.repeat(len(var_1),1)
    #print(yy)
    yy = yy.view(len(var_1),len(var_1))
    #print(yy)
    amat = (xx-yy).abs()
    #print(amat)
    

    xx = var_2.view(-1, 1)
    #print(xx)
    xx = xx.repeat(1, len(var_2))
    #print(xx)
    xx = xx.view(len(var_2),len(var_2))
    #print(xx)
    yy = var_2.repeat(len(var_2),1)
    #print(yy)
    yy = yy.view(len(var_2),len(var_2))
    #print(yy)
    bmat = (xx-yy).abs()
    #print(bmat)

    #print(amat)
    amatavg = torch.mean(amat*normedweight,dim=1)
    #print(amatavg)
    Amat=amat-amatavg.repeat(len(var_1),1).view(len(var_1),len(var_1))\
        -amatavg.view(-1, 1).repeat(1, len(var_1)).view(len(var_1),len(var_1))\
        +torch.mean(amatavg*normedweight)
    #print(amatavg.repeat(len(var_1),1))
    #print(amatavg.repeat(len(var_1),1).view(len(var_1),len(var_1)))
    
    #print(amatavg.view(-1, 1))
    #print(amatavg.view(-1, 1).repeat(1, len(var_1)))
    #print(amatavg.view(-1, 1).repeat(1, len(var_1)).view(len(var_1),len(var_1)))
    
    #print(torch.mean(amatavg*normedweight))
    
    #print(Amat)

    bmatavg = torch.mean(bmat*normedweight,dim=1)
    Bmat=bmat-bmatavg.repeat(len(var_2),1).view(len(var_2),len(var_2))\
        -bmatavg.view(-1, 1).repeat(1, len(var_2)).view(len(var_2),len(var_2))\
        +torch.mean(bmatavg*normedweight)

    #print(Bmat)
    
    ABavg = torch.mean(Amat*Bmat*normedweight,dim=1)
    AAavg = torch.mean(Amat*Amat*normedweight,dim=1)
    BBavg = torch.mean(Bmat*Bmat*normedweight,dim=1)
    #print(ABavg)
    #print(AAavg)
    #print(BBavg)

    if(power==1):
        dCorr=(torch.mean(ABavg*normedweight))/torch.sqrt((torch.mean(AAavg*normedweight)*torch.mean(BBavg*normedweight)))
    elif(power==2):
        dCorr=(torch.mean(ABavg*normedweight))**2/(torch.mean(AAavg*normedweight)*torch.mean(BBavg*normedweight))
    else:
        dCorr=((torch.mean(ABavg*normedweight))/torch.sqrt((torch.mean(AAavg*normedweight)*torch.mean(BBavg*normedweight))))**power
   
    return dCorr




In [5]:
# Modificações feitas por Eduardo

a = torch.tensor([1.0, 2.0, 7.0, 4.0, 5.0])
b = torch.tensor([1.0, 2.0, 1.0, 4.0, 4.0])
n = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0])
p = 1
distance_corr(a, b, n, p)


tensor(0.3662)

In [8]:
a = torch.tensor([1.0, 3.0, 7.0, 4.0, 5.0])
b = torch.tensor([1.0, 2.0, 2.0, 4.0, 1.0])
n = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0])
p = 2

distance_corr(a, b, n, p)

tensor(0.0678)

In [40]:
torch.is_tensor(distance_corr(a, b, n, p))

tensor([[2.0000, 1.4000, 1.2000, 1.4000, 2.0000],
        [2.0000, 1.4000, 1.2000, 1.4000, 2.0000],
        [2.0000, 1.4000, 1.2000, 1.4000, 2.0000],
        [2.0000, 1.4000, 1.2000, 1.4000, 2.0000],
        [2.0000, 1.4000, 1.2000, 1.4000, 2.0000]])


True